# eyeMarks 
> Purpose: Recheck whole-eye mips and establish landmarks for eye  

- [x] Requires: [NAPARI](https://napari.org/) and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html)
- [ ] Intended to exclude eye that are too rotated

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.4
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant
# !pip install napari-clusters-plotter #hdbscan failed
# !pip install napari-plot-profile
# !pip install napari-brightness-contrast
# !pip install napari-curtain
# !pip install napari-3d-ortho-viewer
# !pip install napari-manual-split-and-merge-labels
# !pip install napari-crop
# !pip install napari-stl-exporter

# !conda install -c conda-forge ipywidgets

## to upgrade:
# !pip install cellpose-napari==0.1.4
# !pip install cellpose-napari --upgrade
# !pip install cellpose --upgrade
# !pip install napari --upgrade
# !pip install mxnet-mkl

In [ ]:
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
import ipywidgets
from tqdm import tqdm
import time
import sys
from plotParams import *


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [ ]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

In [ ]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [Define directory paths](#pathNames)
- [Establish eye landmarks](#eyeMarks)
- [Don't forget to update CSV](#csv)

***
# Define pathnames<a id='pathNames' name='pathNames'>∮</a>
***
[Back to Index](#Index)

#### For list of filenanames:

```python
gene = 'nr2e3'
fList = !ls $dPath/*$gene*[002,004].nd2
fList = list(map(lambda st: str.replace(st, dPath + '/', 'fPath = \"'), fList))
fList = list(map(lambda st: str.replace(st, '.nd2', '\"; gene = \'' + gene + '\'; '), fList))
print(*fList, sep = "\n")
```

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()
# data directory
# dPath = "/Volumes/angueyraNEI/LiImaging/zf/20220401_nr2e3F0_s1G_t2T/"
# dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20200821_tbx2a_reImage/"
dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
# analysis directory
# dAnalysis = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/s1Ct2T/"
dAnalysis = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRtbx2F0s/xOGs1C/"

- Use eyeMarks layer to mark:
        - Landmark #1: Center of FOV (where zoom in is centered)
        - Landmark #2: Center of Optic Nerve Head 
        - Landmark #3: Dorsal edge of eye
        - Landmark #4: Ventral edge of eye
        - Landmark #5: Temporal edge of eye (strike zone side)
        - Landmark #6: Nasal edge of eye
- Decide if stack should be excluded (and stop analysis here and make note in csv summary file)

# Establish landmarks in whole-eye MIP

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# open file
dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20200821_tbx2a_reImage/"
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i05b'; gene = 'tbx2a'; # dorsal rotation; excluding.
fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i05d'; gene = 'tbx2a'; # ok
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i06b'; gene = 'tbx2a'; # dorsal rotation; excluding
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i06d'; gene = 'tbx2a'; # still not ideal location
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i07b'; gene = 'tbx2a'; # not ideal either
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i07d'; gene = 'tbx2a'; # good
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i08b'; gene = 'tbx2a'; # borderline
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i08d'; gene = 'tbx2a'; # borderline
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i09b'; gene = 'tbx2a'; # ok
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i09d'; gene = 'tbx2a'; # meh
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i10b'; gene = 'tbx2a'; # good
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i10d'; gene = 'tbx2a'; # good
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i11b'; gene = 'tbx2a';
# fPath = '20200821_5dpf_xOG_s1mCh_CRtbx2a_i11d'; gene = 'tbx2a'; # this is dorsal
fPath = '20200821_5dpf_xOG_s1mCh_uninj_i05b'; gene = 'wt'; # eye marks ok; missing wt data from this day

dPath = "/Volumes/zfSSD/LiImaging/A1R/zf/20201113_CRtbx2b/"
dAnalysis = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRtbx2F0s/xOGs1C/"
# fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i09b'; gene = 'tbx2b'; # looks like a wt; check genotyping
fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i10b'; gene = 'tbx2b';
fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i10d'; gene = 'tbx2b';
fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i12b'; gene = 'tbx2b';
fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i12d'; gene = 'tbx2b';
# fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i13b'; gene = 'tbx2b';
# fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i13d'; gene = 'tbx2b';
# fPath = '20201113_5dpf_xOG_s1mCh_CRtbx2b_i14b'; gene = 'tbx2b';
# fPath = '20201113_5dpf_xOG_s1mCh_uninj_i06b'; gene = 'wt';
# fPath = '20201113_5dpf_xOG_s1mCh_uninjected_i12b'; gene = 'wt';
# fPath = '20201113_5dpf_xOG_s1mCh_uninjected_i12d'; gene = 'wt';
# fPath = '20201113_5dpf_xOG_s1mCh_uninjected_i13b'; gene = 'wt';
# fPath = '20201113_5dpf_xOG_s1mCh_uninjected_i15b'; gene = 'wt';
# fPath = '20201113_5dpf_xOG_s1mCh_uninjected_i15d'; gene = 'wt';

# dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210514_tbx2afoxq2F0/"
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L01b'; gene = 'tbx2a'; # dim eye; looks wt
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L02b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L03b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L04b'; gene = 'tbx2a'; # slightly dorsal but whithin limits
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L05b'; gene = 'tbx2a'; # shows clear lor in extreme phenotype but still milder than tbx2b
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L06b'; gene = 'tbx2a'; # unsure about ONH but looks ok
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L07b'; gene = 'tbx2a'; # this is dorsal; exclude
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L07d'; gene = 'tbx2a'; # clear lor
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L08b'; gene = 'tbx2a'; # not lor
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L08d'; gene = 'tbx2a'; # not lor either
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L09b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L10b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L11b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L11d'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L12b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L13b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L13d'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L14b'; gene = 'tbx2a';
# fPath = '20210514_5dpf_xOG_s1C_CRtbx2a_L15b'; gene = 'tbx2a'; # looks tw

# dAnalysis = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRfoxq2F0/xOGs1C/"
# dPath = "/Volumes/angueyraNEI/LiImaging/A1R/zf2021onwards/20210514_tbx2afoxq2F0/"
# fPath = '20210514_5dpf_xOG_s1C_uninj_L01b'; gene = 'wt';
# #### fPath = '20210514_5dpf_xOG_s1C_uninj_L01b_withDAPI'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L03b'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L04b'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L05b_withDAPI'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L05d'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L06b'; gene = 'wt'; #this is dorsal
# fPath = '20210514_5dpf_xOG_s1C_uninj_L07b_withDAPI'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L09b'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L10b'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L11b'; gene = 'wt';
# fPath = '20210514_5dpf_xOG_s1C_uninj_L12b'; gene = 'wt';

# define analysis directory
dAnalysis = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRlrrfip1aF0s/xOGs1C/"

# open file
dPath = "/Volumes/angueyraNEI/LiImaging/zf/20211215_lrrfip1aF0_xOG_s1C/"
# uninjected
fPath = "20211215_5dpf_xOG_s1mCh_wt_07_002"; gene = 'wt'; # quite curved. good location
fPath = "20211215_5dpf_xOG_s1mCh_wt_08_002"; gene = 'wt'; # quite curved. slight dorsal rotation and rip on other side of zoom in
fPath = "20211215_5dpf_xOG_s1mCh_wt_09_002"; gene = 'wt'; # not bad location

# F0[lrrfip1a]
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_07_002"; gene = 'lrrfip1a'; # this is near ONH but ventral.
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_07_004"; gene = 'lrrfip1a'; # good
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_08_002"; gene = 'lrrfip1a'; # dorsal rotation. EXCLUDE
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_08_004"; gene = 'lrrfip1a'; # very flat; could serve as example; good location too
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_10_002"; gene = 'lrrfip1a';
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_11_002"; gene = 'lrrfip1a'; # curved; could be tricky; bubble inside eye
fPath = "20211215_5dpf_xOG_s1mCh_lrrfip1a_12_002"; gene = 'lrrfip1a'; # also flat. slightly dorsal but whithin limits; bubble inside eye

# define subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
print('Viewer cleared...')


photoreceptors = ['R','U']
colorList = ['green','magenta','bop blue','orange']
# load mips
mipO = 1 #0.65
for pr,cl in zip(photoreceptors,colorList):
    viewer.open(dOut + pr + "_mip.tiff", plugin='builtins', colormap = cl, blending='additive', opacity=mipO);

print('Loaded: ' + fPath)

In [ ]:
nMIP = len(photoreceptors)
for pr in photoreceptors:
    lName = pr + "_mip"
    viewer.layers[lName].scale = [1.5/8,1.5/8]; #viewer.layers[lName].translate = [0,+(512-1024*1.5/(8*2))]
    viewer.layers[lName].translate = [0,1024-(1024*1.5/8)]

# Load whole-eye MIP to establish landmarks
print('Loading eye mip...')
if str.endswith(fPath,'002'):
    mipPath = fPath[:-1] + "1_mip";
elif str.endswith(fPath,'004'):
    mipPath = fPath[:-1] + "3_mip";
elif str.endswith(fPath,'b'):
    mipPath = fPath[:-1] + "a_mip";
elif str.endswith(fPath,'d'):
    mipPath = fPath[:-1] + "c_mip";
viewer.open((dPath+mipPath + ".nd2"))

# viewer.layers.remove(viewer.layers[len(viewer.layers)-1]) # remove transmitted detector image
viewer.layers.select_next();

if len(viewer.layers)==2+nMIP: # no DAPI, GFP and RFP
    nChannels = 2
    viewer.layers[nMIP].colormap = 'green'
    viewer.layers[nMIP].name = 'eyeGreen'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeRed'
elif len(viewer.layers)==3+nMIP: # leave DIC
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'green'
    viewer.layers[nMIP+0].name = 'eyeGreen'
    viewer.layers[nMIP+1].colormap = 'magenta'
    viewer.layers[nMIP+1].name = 'eyeRed'
    viewer.layers[nMIP+2].colormap = 'gray'
    viewer.layers[nMIP+2].name = 'eyeBlue'
    viewer.layers[nMIP+2].opacity = 0.33
elif len(viewer.layers)==4+nMIP: #DIC + DAPI
    nChannels = 3
    viewer.layers[nMIP+0].colormap = 'bop blue'
    viewer.layers[nMIP+0].name = 'eyeBlue'
    viewer.layers[nMIP+1].colormap = 'green'
    viewer.layers[nMIP+1].name = 'eyeGreen'
    viewer.layers[nMIP+2].colormap = 'magenta'
    viewer.layers[nMIP+2].name = 'eyeRed'
    viewer.layers[nMIP+3].colormap = 'gray'
    viewer.layers[nMIP+3].name = 'eyeDIC'
    viewer.layers[nMIP+3].opacity = 0.33

print('Loaded: ' + fPath)
eyeMarks = viewer.add_points(size=20, name = 'eyeMarks', symbol="cross", face_color='#ffffff80')
viewer.layers.select_next();

# Save eye landmarks
eyeMarksButton = ipywidgets.Button(description='Save eyeMarks', layout=ipywidgets.Layout(width='200px', height='100px'))
out = ipywidgets.Output()
def eyeMClick(_):
    with out:
        l = viewer.layers['eyeMarks']; l.save(dOut + l.name + '.csv');
        print('eyeMarks layers saved for ' + fPath)
    # clear viewer
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    eyeMarksButton.disabled = True # to prevent double clicking warnings. All sales are final!
    
eyeMarksButton.on_click(eyeMClick)

print("Make 6 landmark points then click button to save")

ipywidgets.VBox([eyeMarksButton,out])

In [ ]:
l = viewer.layers['eyeMarks']; l.save(dOut + l.name + '.csv');

### [Go back and run next one &uarr;](#zExtract)
---

***
# Quantification<a id='quantification'></a>
***
[Back to Index](#Index)

### Read csv created during analysis and create bar plot

In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/"
# dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_combinedCounts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRtbx2F0s/"
fName = "CRtbx2F0s_combinedCounts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','tbx2a', 'tbx2b']
nGenes = np.size(geneList)
photoreceptors = ['R','U','S','M','L']
# photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        countData = countData[~np.isnan(countData)]
        jitter = np.random.randn(len(countData))*0.03
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png", bbox_inches = "tight")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg", bbox_inches = "tight")

### Stats

### Read csv created during analysis and create bar plot

In [ ]:
geneList = ['wt','nr2e3']
photoreceptors = ['R','U','S','M','L']

print('{0} vs. {1}:'.format(geneList[0],geneList[1]))

for pr in photoreceptors:
    # get counts for each photoreceptor subtype and exclude NaNs
    wtCount = df[(df['genotype']==(geneList[0]))&(df['excludeFlag']==False)][pr]
    wtCount = wtCount[~np.isnan(wtCount)]
    
    crCount = df[(df['genotype']==(geneList[1]))&(df['excludeFlag']==False)][pr]
    crCount = crCount[~np.isnan(crCount)]
    
    u, p = mannwhitneyu(wtCount, crCount)
    print('\t{0}:'.format(pr))
    print('\t\tU = {0:.3f}, p = {1:.5f}, nEyes: wt = {2:.0f}; cr = {3:.0f}'.format(u,p,len(wtCount),len(crCount)))


In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/m2Gs2C/"
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_m2Gs2C_Counts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
# photoreceptors = ['R','U','S','M','L']
photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.01
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg")